In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2024-12-09 15:09:27.700258: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 15:09:27.786948: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 15:09:27.866320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733728167.937125  118523 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733728167.957098  118523 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 15:09:28.127598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# 1. Load dataset
data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# 2. Train base models
svm_model = SVC(probability=True).fit(X_train, y_train)
linear_model = LogisticRegression().fit(X_train, y_train)
tree_model = DecisionTreeClassifier().fit(X_train, y_train)

In [4]:
# Get meta-features
svm_pred = svm_model.predict_proba(X_test)
linear_pred = linear_model.predict_proba(X_test)
tree_pred = tree_model.predict_proba(X_test)

# Stack meta-features
meta_features = np.hstack((svm_pred, linear_pred, tree_pred))
print("Meta-Features Shape:", meta_features.shape)

Meta-Features Shape: (30, 9)


In [5]:
print("y_test Shape:", y_test.shape)

y_test Shape: (30,)


In [6]:
# 3. Train neural network on meta-features
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(meta_features.shape[1],)),  # 更大的隱藏層
    Dense(32, activation='relu'),  # 新增一層隱藏層
    Dense(3, activation='softmax')  # 假設有 3 個類別
])

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(meta_features, y_test, epochs=10, verbose=1)

Epoch 1/10


/home/hank/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1733728170.114333  118523 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step - accuracy: 0.3667 - loss: 1.0382
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7000 - loss: 1.0096
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.9838
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.9585
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.9341
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.9113
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.8897
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.8695
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7000 - loss: 0.8496
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.8305


In [7]:
# 4. Evaluate the model
final_preds = np.argmax(nn_model.predict(meta_features), axis=1)
accuracy = accuracy_score(y_test, final_preds)
print("Stacking Model Accuracy:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Stacking Model Accuracy: 0.7
